# Airbnb Data Warehousing Data Transformation Script

## Set Up

### Install Required Modules

In [1]:
import pandas as pd


### Read in files

In [9]:
reviews = pd.read_csv("data/reviews.csv")
listings = pd.read_csv("data/listings.csv")

### Getting General Information on Listings Table

In [19]:
listings.info(max_cols=None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2668 entries, 0 to 2667
Data columns (total 69 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            2668 non-null   int64         
 1   listing_url                                   2668 non-null   string        
 2   scrape_id                                     2668 non-null   int64         
 3   last_scraped                                  2668 non-null   datetime64[ns]
 4   source                                        2668 non-null   object        
 5   name                                          2668 non-null   object        
 6   neighborhood_overview                         1826 non-null   object        
 7   picture_url                                   2668 non-null   object        
 8   host_id                                       2668 non-null   int64 

### Dropping redundant/empty columns

In [12]:
# These two columns are the same, we can drop one of them
print(pd.to_datetime(listings["last_scraped"]).equals(pd.to_datetime(listings["calendar_last_scraped"])))

# Others have no non-null values, let's remove all the uneccessary fluff

listings = listings.drop("calendar_last_scraped", axis=1)
listings = listings.drop("description",axis=1)
listings = listings.drop("calendar_updated",axis=1)
listings = listings.drop("bedrooms",axis=1)
listings = listings.drop("bathrooms",axis=1)
listings = listings.drop("neighbourhood_group_cleansed",axis=1)



True


### Setting datetime objects to datetime

In [18]:
# Changing the datetime columns to the datetime data type
listings.last_scraped = pd.to_datetime(listings.last_scraped)
listings.host_since = pd.to_datetime(listings.host_since)


## Making all the dimension tables

### Starting off with hosts dimension table

In [24]:
hosts_df = listings[
    [
        "host_id", 
        "host_url", 
        "host_name", 
        "host_since", 
        "host_location", 
        "host_about", 
        "host_response_time",
        "host_response_rate",
        "host_acceptance_rate",
        "host_is_superhost",
        "host_thumbnail_url",
        "host_picture_url",
        "host_neighbourhood",
        "host_listings_count",
        "host_total_listings_count",
        "host_verifications",
        "host_has_profile_pic",
        "host_identity_verified",
        "host_response_time",
        "host_response_rate",
        "host_acceptance_rate",
        "host_is_superhost",
        "host_thumbnail_url",
        "host_picture_url",
        "host_neighbourhood",
        "host_listings_count",
        "host_total_listings_count",
        "host_verifications",
        "host_has_profile_pic",
        "host_identity_verified",
    ]
]